In [72]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.schema import Table, DropTable
import pandas as pd
import os.path

data_sufix = 'data/'

In [73]:
Session = sessionmaker()
engine = create_engine('sqlite:///twitter.db')

In [74]:
hashtags = ['h1b', 'usvisa', 'immigrationreform', 'illegalImmigration', 'immigration',
'immigrantcaravan', 'caravaninvasion', 'buildthewall']
remaining_hashtags = ['illegalaliens', 'birthrightcitizenship']

In [75]:
resTweets = pd.read_sql_query('SELECT hashtag.name AS tag, COUNT(tweet.tweet_id) AS tweets, \
COUNT (DISTINCT user.user_id) AS distinct_tweeting_users \
FROM hashtag \
JOIN tweet ON tweet.hashtag_id = hashtag.id \
JOIN user ON tweet.user_id = user.user_id \
GROUP BY hashtag.name \
ORDER BY COUNT(tweet.tweet_id) DESC', engine)
print(resTweets)

                  tag  tweets  distinct_tweeting_users
0        buildthewall    8608                     3903
1         immigration    3829                     2676
2     caravaninvasion    3172                     1919
3  illegalImmigration     751                      430
4    immigrantcaravan     356                      307
5   immigrationreform     304                      213
6                 h1b     181                       65
7              usvisa      19                       12


In [76]:
resTweets = pd.read_sql_query('SELECT hashtag.name AS tag, COUNT(retweet.retweet_id) AS retweets \
FROM hashtag \
JOIN tweet ON tweet.hashtag_id = hashtag.id \
JOIN retweet ON retweet.source_tweet_id = tweet.tweet_id \
GROUP BY hashtag.name \
ORDER BY COUNT(retweet.retweet_id) DESC', engine)
print(resTweets)

                  tag  retweets
0        buildthewall     54934
1     caravaninvasion     14814
2         immigration      9148
3  illegalImmigration      2680
4    immigrantcaravan      1675
5   immigrationreform       422
6                 h1b       213
7              usvisa         5


In [77]:
resDatesTags = pd.read_sql_query('SELECT hashtag.name AS tag, DATE(tweet_created_at) AS date, \
COUNT(DISTINCT tweet_id) AS tweets_per_day \
FROM hashtag \
JOIN tweet ON tweet.hashtag_id = hashtag.id \
GROUP BY hashtag.name, DATE(tweet_created_at)', engine)
print(resDatesTags)

                   tag        date  tweets_per_day
0         buildthewall  2018-10-27            1474
1         buildthewall  2018-10-28            1703
2         buildthewall  2018-10-29            2589
3         buildthewall  2018-10-30            2842
4      caravaninvasion  2018-10-27             594
5      caravaninvasion  2018-10-28             492
6      caravaninvasion  2018-10-29             941
7      caravaninvasion  2018-10-30            1145
8                  h1b  2018-10-27              33
9                  h1b  2018-10-28              26
10                 h1b  2018-10-29              56
11                 h1b  2018-10-30              66
12  illegalImmigration  2018-10-27             116
13  illegalImmigration  2018-10-28             119
14  illegalImmigration  2018-10-29             208
15  illegalImmigration  2018-10-30             308
16    immigrantcaravan  2018-10-27              58
17    immigrantcaravan  2018-10-28              48
18    immigrantcaravan  2018-10

In [78]:
resDates = pd.read_sql_query('SELECT DATE(tweet_created_at) AS date, \
COUNT(DISTINCT tweet.tweet_id) AS tweets_per_day, \
COUNT(DISTINCT user.user_id) AS distinct_users_per_day \
FROM tweet \
JOIN user ON tweet.user_id = user.user_id \
GROUP BY DATE(tweet_created_at)', engine)
print(resDates)

         date  tweets_per_day  distinct_users_per_day
0  2018-10-27            2779                    1868
1  2018-10-28            2803                    1819
2  2018-10-29            4783                    3027
3  2018-10-30            6079                    3957


In [79]:
# Korelacja pomiędzy followers_count usera a łączną liczbą retweetów postów usera
# Wyłonienie najbardziej wpływowych użytkowników sieci

resRetweetUserFriends = pd.read_sql_query('SELECT COUNT(DISTINCT retweet.retweet_id) AS total_retweets, \
COUNT(DISTINCT tweet.tweet_id) AS tweets_got_retweeted, \
user.followers_count AS followers, user.user_name as user \
FROM retweet \
INNER JOIN tweet ON retweet.source_tweet_id = tweet.tweet_id \
INNER JOIN user ON tweet.user_id = user.user_id \
GROUP BY(user.user_id) \
ORDER BY COUNT(DISTINCT retweet.retweet_id) DESC \
LIMIT 10', engine)
print(resRetweetUserFriends)

   total_retweets  tweets_got_retweeted  followers                        user
0            7852                     5      92129                Patriot Gary
1            6489                     1      66644                   TheKat ❤️
2            5107                     4      34121                        Cody
3            3303                    53      21566    fordnFLA777 (K)♥️✝️ ❤️✡️
4            1717                     3      73262            🇺🇸 SKY45 🇺🇸 ︻デ═一
5            1538                    27      20299       GO RED STATE BY STATE
6            1461                     2     178318    🦅 🦅 Nationalist Eagle 🦅🦅
7            1412                     3      25052            Jennifer 🔱🇺🇸🔱🌟🌟🌟
8            1192                     1      34039  Chris  🇺🇸 Let Freedom Ring
9            1127                     3      58527              Laura Vote Red
